In [3]:
import pandas as pd
from pandas_datareader import data as wb

df = wb.DataReader(["VALE3.SA", "PETR4.SA", "VVAR3.SA", "RAIL3.SA",
                   "MGLU3.SA", "LAME4.SA", "JHSF3.SA", "JBSS3.SA",
                   "ENGI11.SA", "DIRR3.SA", "CCRO3.SA", "BEEF3.SA",
                   "BBDC4.SA"], 
                   data_source='yahoo', start='1-1-2015')
df = df["Adj Close"]

In [4]:
x = wb.DataReader(["ITUB4.SA"], data_source='yahoo', start='1-1-2015')
x = x["Adj Close"]


## Projection Pricing 
* Nesse notebook serão implementados as formas de precificação apresentadas por Luenberger no seu Paper "Projection Pricing".

Resumo: Um problema em finanças é a precificar ativos os quais não são combinações lineares de ativos já precificados. 
No artigo, Luenberger ataca esse problema de forma geométrica. O preço pelo qual se vende um ativo é visto como um vetor ($y_k$) em um Espaço de Hilbert ($H$). Os ativos já precificados formam um subespaço vetorial ($M \subset H$), no qual é definido um funcional linear ($f:M\rightarrow\mathbb{R}$) que precifica o ativo (preço pelo qual se compra).

Exemplificando: 
<center>   
    $M = span\{y_1, y_2, y_3, ..., y_n\}$
</center> 
Onde M são os ativos já comercializados. 

Se $x \in M$: 
<center>
    $x = \omega_1 y_1+\omega_2 y_2+...+\omega_n y_n$
</center>
Logo, o preço de $x$ será: 
<center>
    $f(x) = \omega_1 p_1+\omega_2 p_2+...+\omega_n p_n$ 
</center>
O produto interno defindo em $H$ é:
<center>
    $(x|y) = \mathbb{E}[xy] = cov(x, y)+\mathbb{E}[x]\mathbb{E}[y] $ 
</center>

Serão implementados, em ordem, 

1. [ ] Standard Projection
    * [ ] Orthogonal Extension 
    * [ ] Covariance Form 
    * [ ] Minimum Norm Pricing 
2. [ ] Minimum  Norm Formulation 
3. [ ] Optimal Portfolio Formulation


                                


## Standard Projection
Essa é a forma mais básica e intuitiva. Seja $x\in H$ um ativo que está fora de $M$. A ideia é de projetar $x$ para $M$ e o seu preço será igual ao da projeção. Se $m_x = \omega_1 y_1+\omega_2 y_2+...+\omega_n y_n = \omega^Ty$ é a projeção, $x- m_x$ deverá ser perpendicular a M, logo: 

<center>
    $(x-m_x|y_k) = 0$ para todo k.
</center>

Matricialmente teremos: 
<center>
   $\begin{bmatrix}
    (y_1|y_1) & (y_1|y_2) & \cdots & (y_1|y_n)\\
    (y_2|y_1) & (y_2|y_2) & \cdots & (y_1|y_n)\\
    \vdots    &   \vdots  & \ddots & \vdots \\
    (y_n|y_1) & (y_n|y_2) & \cdots & (y_n|y_n)\\
    \end{bmatrix}$
    $\begin{bmatrix}
    \omega_1\\
    \omega_2\\
    \vdots\\
    \omega_n
    \end{bmatrix}$ = 
    $\begin{bmatrix}
    (y_1|x)\\
    (y_2|x)\\
    \vdots\\
    (y_n|x)
    \end{bmatrix}$
   
</center>

Resolvendo o sistema, podemos encontrar o vetor de pesos $\omega$:

<center>
    $\omega =  Y^{-1} b$
</center>
E precificá-lo: 
<center>
    $p_x =\omega^T p$
</center>

### Código
Vamos supor que recebemos a série de todos os ativos já negociados. 

A matriz $Y$ pode calculada como:
<center>
    $Y = V + \bar{y}\bar{y}^T$
</center>
Onde $\bar{y}$ é o vetor de "Expected Payoffs" e V a matriz de Covariância.

In [2]:
import numpy as np 
import pandas as pd 
class StandardProjection():
    def __init__(self):
        pass
    
    def fit(self, df):
        self.df = df.copy()
        self.V = np.array(df.cov())
        self.y_bar = np.array((df).mean()).reshape(len(df.columns), 1)
        self.Y = self.V + self.y_bar@self.y_bar.T
        self.p = np.array(df.iloc[len(df)-1]).reshape(len(df.columns), 1)
        
    def price(self, x):
        aux = self.df.copy()
        aux["pricing"] = x
        cov = np.array(aux.cov()) #Obtem a Covariancia de "x" com todos os outros vetores.
        b = cov[: cov.shape[0]-1, cov.shape[1]-1].reshape(cov.shape[1]-1, 1)+aux["pricing"].mean()*self.y_bar
        omega = np.linalg.inv(self.Y)@b
        return {"Preço: ": float(omega.T@self.p), "w: ": omega}

In [6]:
test = StandardProjection()
test.fit(df)
test.price(x)

{'Preço: ': 27.061610509143364,
 'w: ': array([[ 0.08781788],
        [ 0.07951123],
        [ 0.07203052],
        [ 0.05064967],
        [-0.25755509],
        [ 0.10467578],
        [-0.31203315],
        [-0.08001048],
        [ 0.14107868],
        [-0.20255175],
        [ 0.04839294],
        [ 0.08530794],
        [ 0.79039804]])}

## Orthogonal Extension
Essa forma é uma reorganização da anterior. Não há nenhuma mudança na ideia, apenas uma reinterpretação.
Podemos escrever, $p_x$ da seguinte forma:

<center>
    $p_x =\textbf{g}^Tb$
</center>
Onde $\textbf{g} = Y^{-1}\textbf{p}$. Então:
<center>
    $p_x = \mathbb{E}[\textbf{g}^T\textbf{y}x] = (\textbf{g}^T\textbf{y}|x) = (g|x)$
</center>
Isso mostra que o funcional de precificação pode ser extendido para todo o espaço $H$ como um produto interno de um certo vetor $g\in M$. Mais geralmente, esse vetor é o vetor de minima norma que tem preço igual a $||f||^2$.

Então, para encontrarmos $g$, basta resolvermos o seguinte problema de Otimização:
<center>
  $min_{\textbf{g}}$ $\mathbb{E}[\textbf{g}^Tyy^T\textbf{g}]$<br>  
  $subject$ $to$ $\textbf{g}^T\textbf{p} = ||f||^2$
</center>

Que pode ser reorganizado da seguinte forma:
<center>
  $min_{\textbf{g}}$ $\textbf{g}^T Y \textbf{g}$<br>  
  $subject$ $to$ $\textbf{g}^T\textbf{p} = ||f||^2$
</center>
Para simplificar mais ainda, podemos restringir o preço de $\textbf{g}$ para ser um, encontrando um vetor $g'$ e depois precificar algo que ja conhecemos para obter a constante de proporcionalidade. Ou seja:

<center>
  $min_{\textbf{g'}}$ $\textbf{g'}^T Y \textbf{g'}$<br>  
  $subject$ $to$ $\textbf{g'}^T\textbf{p} = 1$
</center>
E ai pegamos algo que já conhecemos o preço e ajustamos:
<center>
    $(\alpha g'|y_1)= p_1$
</center> 
Onde poderemos encontrar o alpha.

In [36]:
class OrthogonalExtension:
    def __init__():
        pass
    
    def fit():
        self.df = df.copy()
        self.V = np.array(df.cov())
        self.y_bar = np.array((df).mean()).reshape(len(df.columns), 1)
        self.Y = self.V + self.y_bar@self.y_bar.T
        self.p = np.array(df.iloc[len(df)-1]).reshape(len(df.columns), 1)
        
        def objective(Y):
            return lambda g: float(g.T@Y@g)
        
        def constraint(p):
            return lambda g: g.T@p-1
        
        g0 = np.random.randn(len(self.df.columns)).reshape(len(self.df.columns), 1)
        
        con = {"type": "eq", "fun": constraint(np.array([[1], [1]]))}
        
        sol = minimize(objective(Y), g0, method ="SLSQP", constraints = con)
        
        x = sol.x
        
    def price():
        pass

In [42]:
df[["VALE3.SA", "PETR4.SA"]]

Symbols,VALE3.SA,PETR4.SA
Date,,
2015-01-02,17.272884,8.683292
2015-01-05,17.013140,7.941133
2015-01-06,17.694962,7.681376
2015-01-07,18.344318,8.043180
2015-01-08,18.539125,8.562694
...,...,...
2021-02-25,95.709999,23.190001
2021-02-26,94.519997,22.240000
2021-03-01,98.570000,22.000000


In [50]:
np.array(df[["VALE3.SA", "PETR4.SA"]].mean()).reshape(2,1).T@np.array([[1], 
                                        [2]])

array([[69.96165862]])

In [33]:
from scipy.optimize import minimize

def objective(Y):
    return lambda g: float(g.T@Y@g)

def constraint(p):
    return lambda g: g.T@p-1

con ={"type": "eq", "fun": constraint(np.array([[1], [1]]))}

g0 = np.array([[0.25], [0.5]])
sol = minimize(objective(Y), g0, method ="SLSQP", constraints = con)
sol

     fun: 0.24000000000000085
     jac: array([0.48000004, 0.48000002])
 message: 'Optimization terminated successfully'
    nfev: 10
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([-1.,  2.])

In [28]:
ybar= np.array([[1.4], [0.8]])
global V 
V = np.array([[0.04, 0], [0, 0.04]])
global Y
Y = V + ybar@ybar.T

In [29]:
Y

array([[2.  , 1.12],
       [1.12, 0.68]])